In [1]:
import pandas as pd

# Path relative to your notebook location
fake_df = pd.read_csv('../data/raw/Fake.csv')
true_df = pd.read_csv('../data/raw/True.csv')

print(f"Fake news shape: {fake_df.shape}")
print(f"True news shape: {true_df.shape}")

Fake news shape: (23481, 4)
True news shape: (21417, 4)
